In [1]:
import os
import csv
import time
import glob
import csv
import glob
import shutil
import librosa
import argparse
import warnings
import torch
from pathlib import Path
import transformers
print(transformers.__version__)
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
import warnings
warnings.filterwarnings("ignore")

4.39.3


2024-05-20 08:37:12.590411: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-20 08:37:12.590676: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-20 08:37:12.758247: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
MODEL = '/kaggle/input/bengali-ai-asr-submission/bengali-whisper-medium/'
PUNCT_MODELS = [
    '/kaggle/input/bengali-ai-asr-submission/punct-model-6layers/',
    '/kaggle/input/bengali-ai-asr-submission/punct-model-8layers/',
    '/kaggle/input/bengali-ai-asr-submission/punct-model-11layers/',
    '/kaggle/input/bengali-ai-asr-submission/punct-model-12layers/'
]
CHUNK_LENGTH_S = 20.1
ENABLE_BEAM = True
PUNCT_WEIGHTS = [[1.0, 1.4, 1.0, 0.8]]
BATCH_SIZE = 4
EVAL = True
pipe = pipeline(task="automatic-speech-recognition",
                model=MODEL,
                tokenizer=MODEL,
                chunk_length_s=CHUNK_LENGTH_S, batch_size=BATCH_SIZE)
pipe.model.config.forced_decoder_ids = pipe.tokenizer.get_decoder_prompt_ids(language="bn", task="transcribe")
models = [
    AutoModelForTokenClassification.from_pretrained(f).eval().cpu() for f in PUNCT_MODELS
]
tokenizer = AutoTokenizer.from_pretrained(PUNCT_MODELS[0])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def fix_repetition(text, max_count):
    uniq_word_counter = {}
    words = text.split()
    for word in text.split():
        if word not in uniq_word_counter:
            uniq_word_counter[word] = 1
        else:
            uniq_word_counter[word] += 1

    for word, count in uniq_word_counter.items():
        if count > max_count:
            words = [w for w in words if w != word]
    text = " ".join(words)
    return text

def punctuate(text):
    input_ids = tokenizer(text).input_ids
    with torch.no_grad():
        model = models[0]
        logits = torch.nn.functional.softmax(
            model(input_ids=torch.LongTensor([input_ids]).cpu()).logits[0, 1:-1],
            dim=1).cpu()
        for model in models[1:]:
            logits += torch.nn.functional.softmax(
                model(input_ids=torch.LongTensor([input_ids]).cpu()).logits[0, 1:-1],
                dim=1).cpu()
        logits = logits / len(models)
        logits *= torch.FloatTensor(PUNCT_WEIGHTS)
        label_ids = torch.argmax(logits, dim=-1)

        tokens = tokenizer(text, add_special_tokens=False).input_ids
        punct_text = ""
        for index, token in enumerate(tokens):
            token_str = tokenizer.decode(token)
            if '##' not in token_str:
                punct_text += " " + token_str
            else:
                punct_text += token_str[2:]
            punct_text += ['', '।', ',', '?'][label_ids[index].item()]

    punct_text = punct_text.strip()
    return punct_text

In [4]:
def prediction(location):
    files = [
    location
    ]
    texts = pipe(files)
    for f, text in zip(files, texts):
        file_id = Path(f).stem
        pred = text['text'].strip()
        pred = fix_repetition(pred, max_count=8)
        pred = punctuate(pred)
        if pred[-1] not in ['।', '?', ',']:
            pred = pred + '।'
        # print(i, file_id, pred)
        return pred

In [13]:
location = '/kaggle/input/recordings/Standard recording 3.mp3'

In [14]:
from pydub import AudioSegment

display(AudioSegment.from_file(location))
prediction(location)

'সকালবেলা একটা মোবাইল পাওয়া গেছে। আব্বুর মোবাইল পাওয়া গেছে।'